#### [ 동적 모델 설계 구현 ]

- 사용 모듈 : nn.ModuleList 
- 특징 : 일반 list로는 pytorch에서 layer 인식 안됨! ==> 대안 : ModuleList


In [1]:
## -------------------------------------------------------------------
## 모델 설계
## 입력층
## 은닉층  <-- 유동적 0개 ~ N개 : 모델 인스턴스 생성 시 매개변수 전달 인자 
## 출력층
## -------------------------------------------------------------------

In [2]:
## 모듈 로딩
import torch 
import torch.nn as nn 
import torch.nn.functional as F

In [ ]:
## 동적 모델 클래스 정의 (1)
class MyModel(nn.Module):

    ## 동적 층 구성 초기화 메서드 
    def __init__(self, in_in, out_out, h_in, h_cnt):
        super().__init__()
        # 입력층
        self.input_layer= nn.Linear(in_in, h_in)
        # 은닉층
        self.h1_layer = nn.ModuleList( [ nn.Linear(h_in, h_in) for _ in range(h_cnt)])
        # 출력층
        self.output_layer = nn.Linear(h_in, out_out) 

    

In [ ]:
## 동적 모델 클래스 정의 (1) - 은닉층 개수 동적 
class MyModel(nn.Module):

    ## 동적 층 구성 초기화 메서드 
    def __init__(self, in_in, out_out, h_in, h_cnt):
        super().__init__()
        # 입력층
        self.input_layer= nn.Linear(in_in, h_in)
        # 은닉층
        self.h1_layer = nn.ModuleList( [ nn.Linear(h_in, h_in) for _ in range(h_cnt)])
        # 출력층
        self.output_layer = nn.Linear(h_in, out_out) 

    
    ## 순전파 학습
    def forward(self, x):

        out = F.relu(self.input_layer(x))

        for hidden in self.h1_layer:
            out = F.relu(hidden(out))

        return self.output_layer(out)

In [19]:
from torchinfo import summary

In [ ]:
## 모델 인스턴스 생성 및 테스트 
# in_in  = 3   # out_out= 1  # hd_in  = 5   # hd_cnt = 1
m1 = MyModel(3, 1, 5, 1)
m1(torch.FloatTensor([[1,2,3]]))     # (샘플수, 피쳐수) 

summary(m1, input_size=(100, 3))

Layer (type:depth-idx)                   Output Shape              Param #
MyModel                                  [100, 1]                  --
├─Linear: 1-1                            [100, 5]                  20
├─ModuleList: 1-2                        --                        --
│    └─Linear: 2-1                       [100, 5]                  30
├─Linear: 1-3                            [100, 1]                  6
Total params: 56
Trainable params: 56
Non-trainable params: 0
Total mult-adds (M): 0.01
Input size (MB): 0.00
Forward/backward pass size (MB): 0.01
Params size (MB): 0.00
Estimated Total Size (MB): 0.01

In [22]:
# in_in  = 3   # out_out= 3  # hd_in  = 10   # hd_cnt = 5
m2 = MyModel(3, 3, 10, 5)
m2(torch.FloatTensor([[1,2,3]]))     # (샘플수, 피쳐수) 

summary(m2, input_size=(100, 3))

Layer (type:depth-idx)                   Output Shape              Param #
MyModel                                  [100, 3]                  --
├─Linear: 1-1                            [100, 10]                 40
├─ModuleList: 1-2                        --                        --
│    └─Linear: 2-1                       [100, 10]                 110
│    └─Linear: 2-2                       [100, 10]                 110
│    └─Linear: 2-3                       [100, 10]                 110
│    └─Linear: 2-4                       [100, 10]                 110
│    └─Linear: 2-5                       [100, 10]                 110
├─Linear: 1-3                            [100, 3]                  33
Total params: 623
Trainable params: 623
Non-trainable params: 0
Total mult-adds (M): 0.06
Input size (MB): 0.00
Forward/backward pass size (MB): 0.05
Params size (MB): 0.00
Estimated Total Size (MB): 0.05

In [23]:
## 동적 모델 클래스 정의 (2) - 은닉층 개수 및 뉴런 개수 동적인 모델 -------------------
## in_in   : 입력층의 입력   in_out    : 입력층의 출력 / 다음층의 입력
## out_out : 출력층의 출력   h_outs=[] : 은닉층의 출력 리스트
class DynamicModel(nn.Module):
    def __init__(self, in_in, in_out, out_out, h_outs=[]):
        # 부모클래스 생성
        super().__init__()
        # 입력층 생성
        self.input_layer= nn.Linear( in_in,  in_out ) 

        # 은닉층 여러개 생성
        self.h1_layer=nn.ModuleList()
        for idx in range(len(h_outs)):  
            h_in  = h_outs[idx-1] if idx else  in_out
            h_out = h_outs[idx]
            self.h1_layer.append( nn.Linear(h_in, h_out) )

        # 출력층 생성
        self.output_layer=nn.Linear( h_outs[-1] if len(h_outs) else in_out,  out_out )

    # 순전파 학습 메서드 -------------------------------------------------
    def forward(self, x):
        y=F.relu(self.input_layer(x))
        
        for linear in self.h1_layer: 
            y=F.relu(linear(y))
            
        return self.output_layer(y)
    


In [24]:
## 모델 인스턴스 생성 및 테스트 
## in_in   : 입력층의 입력   in_out    : 입력층의 출력 / 다음층의 입력
## out_out : 출력층의 출력   h_outs=[] : 은닉층의 출력 리스트
m1 = DynamicModel(3, 5, 1)
m1(torch.FloatTensor([[1,2,3]]))     # (샘플수, 피쳐수) 

summary(m1, input_size=(100, 3))

Layer (type:depth-idx)                   Output Shape              Param #
DynamicModel                             [100, 1]                  --
├─Linear: 1-1                            [100, 5]                  20
├─Linear: 1-2                            [100, 1]                  6
Total params: 26
Trainable params: 26
Non-trainable params: 0
Total mult-adds (M): 0.00
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.01

In [25]:
## 모델 인스턴스 생성 및 테스트 
## in_in   : 입력층의 입력   in_out    : 입력층의 출력 / 다음층의 입력
## out_out : 출력층의 출력   h_outs=[] : 은닉층의 출력 리스트
m1 = DynamicModel(3, 10, 3, [7, 5])
m1(torch.FloatTensor([[1,2,3]]))     # (샘플수, 피쳐수) 

summary(m1, input_size=(100, 3))

Layer (type:depth-idx)                   Output Shape              Param #
DynamicModel                             [100, 3]                  --
├─Linear: 1-1                            [100, 10]                 40
├─ModuleList: 1-2                        --                        --
│    └─Linear: 2-1                       [100, 7]                  77
│    └─Linear: 2-2                       [100, 5]                  40
├─Linear: 1-3                            [100, 3]                  18
Total params: 175
Trainable params: 175
Non-trainable params: 0
Total mult-adds (M): 0.02
Input size (MB): 0.00
Forward/backward pass size (MB): 0.02
Params size (MB): 0.00
Estimated Total Size (MB): 0.02

In [26]:
for name, param in m1.named_parameters():
    print(f'[{name}]-------------------\n{param}')

[input_layer.weight]-------------------
Parameter containing:
tensor([[-0.3934, -0.0091, -0.2757],
        [ 0.0559, -0.4245,  0.5056],
        [ 0.5641, -0.1437,  0.4157],
        [-0.5158,  0.2054, -0.0737],
        [ 0.2024, -0.2727, -0.1350],
        [-0.1544,  0.1046,  0.3444],
        [-0.2588, -0.5181,  0.3021],
        [-0.5072, -0.3291,  0.3278],
        [-0.3733,  0.0798,  0.4676],
        [ 0.3100, -0.0990,  0.0575]], requires_grad=True)
[input_layer.bias]-------------------
Parameter containing:
tensor([ 0.3346,  0.3254,  0.1174,  0.3424,  0.3860, -0.0801,  0.4711, -0.0054,
        -0.2339, -0.1561], requires_grad=True)
[h1_layer.0.weight]-------------------
Parameter containing:
tensor([[ 0.0963,  0.1053,  0.0207,  0.2670, -0.0724, -0.2170,  0.1538, -0.0899,
         -0.2465,  0.1174],
        [ 0.2722,  0.1705, -0.1623,  0.1700, -0.0993, -0.2354, -0.1813, -0.2854,
         -0.0235, -0.2137],
        [ 0.2249,  0.2810, -0.2455, -0.1945,  0.2742, -0.3113,  0.3123,  0.0413,


In [ ]:
class MyModule(nn.Module):
        def __init__(self):
            super().__init__()
            self.choices = nn.ModuleDict({
                    'conv': nn.Conv2d(10, 10, 3),
                    'pool': nn.MaxPool2d(3)
            })
            self.activations = nn.ModuleDict([
                    ['lrelu', nn.LeakyReLU()],
                    ['prelu', nn.PReLU()]
            ])

        def forward(self, x, choice, act):
            x = self.choices[choice](x)
            x = self.activations[act](x)
            return x